# 1. Imports and Loading the Network

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
G = nx.read_edgelist("data/sbb.edgelist", delimiter=";", create_using=nx.Graph)

connected_comp = nx.connected_components(G)
max_connected_comp = max(connected_comp)
G = G.subgraph(max_connected_comp)

# 2. Benchmark: Iteration Time, Silhouette Index and Modularity

In [3]:
from sklearn.metrics import silhouette_score
import time
import os

# create directory to save benchmarks results
if not os.path.exists("data/benchmarks"):
    os.makedirs("data/benchmarks")


def multi_iter_benchmark(G, community_detec_func, iters=10, time_threshold=12, leiden=False):
    times = []
    scores_hops = []
    scores_coords = []
    modularities = []
    
    for _ in range(iters):
        partitions = community_detec_func(G)
        t, s_hops, s_coords, m = single_iter_benchmark(G, partitions, time_threshold, leiden)
        times.append(t)
        scores_hops.append(s_hops)
        scores_coords.append(s_coords)
        modularities.append(m)

    max_len_iteration = max([len(t) for t in times])
    
    for (t, s_hops, s_coords, m) in zip(times, scores_hops, scores_coords, modularities):
        while len(t) < max_len_iteration:
            t.append(np.nan)
            s_hops.append(np.nan)
            s_coords.append(np.nan)
            m.append(np.nan)

    mean_times = np.nanmean(times, axis=0)
    mean_scores_hops = np.nanmean(scores_hops, axis=0)
    mean_scores_coords = np.nanmean(scores_coords, axis=0)
    mean_modularities = np.nanmean(modularities, axis=0)

    return mean_times, mean_scores_hops, mean_scores_coords, mean_modularities

# iteration benchmarking function
# you must give it a partition scheme before entering the function
# time_threshold is the maximum number of hours the benchmark should run for. Useful for Girvan-Newman
def single_iter_benchmark(G, partitions, time_threshold=12, leiden=False):
    timeout = time.time() + 60 * 60 * time_threshold

    i_time = []
    i_silhouette_hops = []
    i_silhouette_coords = []
    i_modularity = []
    
    while True:
        # Timeout necessary for Girvan Newman
        if time.time() > timeout:
            break
            
        start = time.time()
        communities = next(partitions, 'finished')
        stop = time.time()
        
        if communities == 'finished':
            break
        
        time_diff = stop - start
        i_time.append(time_diff)

        # Necessary for leiden: convert node ids to node names
        if leiden == True:
            communities = convert_leiden(G, communities)
        
        i_silhouette_hops.append(silhouette_hops(G, communities))
        i_silhouette_coords.append(silhouette_coordinates(G, communities))
        i_modularity.append(nx.community.modularity(G, communities))
        
        final_communities = communities
        
    return i_time, i_silhouette_hops, i_silhouette_coords, i_modularity


# silhouette score function
def silhouette_hops(G, found_communities):
    # make a dictionary with node as key and final community as value
    node_to_community = {node: comm_idx for comm_idx, community in enumerate(found_communities) for node in community}
    # list all final communities by indexing with keys
    comm_labels = [node_to_community[node] for node in sorted(G.nodes())]
    
    # calc silhouette score by hop matrix as an array and the communities as labels
    sil_score = silhouette_score(hop_mat, comm_labels, metric='precomputed', random_state=42)

    return sil_score

# silhouette score function
def silhouette_coordinates(G, found_communities):
    # make a dictionary with node as key and final community as value
    node_to_community = {node: comm_idx for comm_idx, community in enumerate(found_communities) for node in community}
    # list all final communities by indexing with keys
    comm_labels = [node_to_community[node] for node in sorted(G.nodes())]
    
    # calc silhouette score by inputting coordinate matrix as an array and the communities as labels
    sil_score = silhouette_score(coordinates_mat, comm_labels, metric='euclidean', random_state=42)

    return sil_score

# create the coordinate matrix
stations_gdf = gpd.read_file('data/stations.geojson')
stations_gdf = stations_gdf[stations_gdf["station_name"].isin(list(G.nodes))] # make sure that G.nodes is the Largest Connected Component Graph
coordinates_mat = np.column_stack((stations_gdf["geometry"].x, stations_gdf["geometry"].y))

# create the hop matrix
length = dict(nx.all_pairs_shortest_path_length(G))
df = pd.DataFrame(length)
df = df.sort_index().sort_index(axis=1)
hop_mat = df.to_numpy()

## 2.1 Louvain

In [4]:
t, s_hops, s_coords, m = multi_iter_benchmark(G, nx.community.louvain_partitions)

louvain_bench = pd.DataFrame(zip(t, s_hops, s_coords, m), columns=["time", "silhouette hops", "silhouette coordinates", "modularity"])
louvain_bench.to_csv("./data/benchmarks/louvain.csv", index=False)
louvain_bench

,time,silhouette hops,silhouette coordinates,modularity
0,0.327464,0.344530,0.140315,0.561615
1,0.085966,0.310990,0.046505,0.817479
2,0.028904,0.220866,-0.070725,0.895794
3,0.010234,0.147164,-0.046892,0.906167
4,0.004340,0.144790,-0.034408,0.906612


## 2.2 Louvain Implementation

In [12]:
from louvain import louvain_generator
# remade the graph due to a frozen graph error
G = nx.read_edgelist("data/sbb.edgelist", delimiter=";", create_using=nx.Graph)
connected_comp = nx.connected_components(G)
max_connected_comp = max(connected_comp)
sub_G = G.subgraph(max_connected_comp)
G = nx.Graph(sub_G)

t, s_hops, s_coords, m = multi_iter_benchmark(G, louvain_generator)

louvain_imp_bench = pd.DataFrame(zip(t, s_hops, s_coords, m), columns=["time", "silhouette hops", "silhouette coordinates", "modularity"])
louvain_imp_bench.to_csv("./data/benchmarks/louvain_imp.csv", index=False)
louvain_imp_bench

,time,silhouette hops,silhouette coordinates,modularity
0,4.637026,0.344104,0.139619,0.556008
1,0.513469,0.278283,0.032737,0.777300
2,0.201005,0.215072,-0.096429,0.872622
3,0.067714,0.125470,-0.140040,0.900634


## 2.3 Girvan-Newman

In [17]:
gn_partitions = nx.community.girvan_newman(G)
t, s_hops, s_coords, m = single_iter_benchmark(G, gn_partitions, time_threshold=0.75)

gn_bench = pd.DataFrame(zip(t, s_hops, s_coords, m), columns=["time", "silhouette hops", "silhouette coordinates", "modularity"])
gn_bench.to_csv("./data/benchmarks/girvan-newman.csv", index=False)

ValueError: 3 columns passed, passed data had 4 columns

## 2.4 Leiden

In [6]:
import leidenalg as la
import igraph as ig


def convert_leiden(G, partition):
    idx2name = {idx: name for idx, name in enumerate(list(G.nodes))}
    
    communities = []
    for comm in partition:
        communities.append({idx2name[n] for n in comm})
        
    return communities


def leiden_gen(G):
    graph = ig.Graph.from_networkx(G, vertex_attr_hashable='name')

    # create the partition with every node as its community
    partition = la.ModularityVertexPartition(graph)
    optimiser = la.Optimiser()
        
    diff = 1
    while diff > 0:
        diff = optimiser.optimise_partition(partition)
        yield partition

In [7]:
t, s_hops, s_coords, m = multi_iter_benchmark(G, leiden_gen, leiden=True)

leiden_bench = pd.DataFrame(zip(t, s_hops, s_coords, m), columns=["time", "silhouette hops", "silhouette coordinates", "modularity"])
leiden_bench.to_csv("./data/benchmarks/leiden.csv", index=False)
leiden_bench

,time,silhouette hops,silhouette coordinates,modularity
0,0.111491,0.149870,-0.010167,0.908269
1,0.016735,0.148954,0.002682,0.909099
2,0.018227,0.148926,0.006339,0.909476
3,0.020094,0.149374,0.009630,0.909565
4,0.020508,0.148370,0.013778,0.909714
5,0.017632,0.147884,0.020522,0.909911
6,0.018069,0.149625,0.025750,0.909934
7,0.013462,0.144547,0.043134,0.909866
8,0.018253,0.144547,0.043134,0.909866


# 3. Evaluation of the Community Detection Algorithms

In [13]:
def plot_bench_results(df_list, labels):
    fig, axs = plt.subplots(nrows=1, ncols=len(df_list[0].columns), figsize=(18, 6), layout="tight")
    # iterate over the columns
    for i, col in enumerate(df_list[0].columns):
        # iterate over the dataframe: all in the same plot
        for df, label in zip(df_list, labels):
            axs[i].plot(df[col], label=label)
        
        axs[i].set_xlabel('Iteration')
        axs[i].set_ylabel(col)
        axs[i].set_title(f'{col} v. Iteration')
            # force int on x-axis
        axs[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        axs[i].legend()
    
    plt.savefig("community-detection-benchmarks.png")
    plt.show()

In [ ]:
louvain_bench = pd.read_csv("data/benchmarks/louvain.csv")
louvain_imp_bench = pd.read_csv("data/benchmarks/louvain_imp.csv")
gn_bench = pd.read_csv("data/benchmarks/girvan-newman.csv")
leiden_bench = pd.read_csv("data/benchmarks/leiden.csv")

algorithm_data = [louvain_bench, louvain_imp_bench, leiden_bench, gn_bench[:9]]
algorithm_name = ["Louvain", "Louvain Implementation", "Leiden", "Girvan-Newman"]

# algorithm_data = [louvain_bench, louvain_imp_bench, leiden_bench]
# algorithm_name = ["Louvain", "Louvain Implementation", "Leiden"]

plot_bench_results(algorithm_data, algorithm_name)

In [ ]:
# we should be able to safely remove this
# kept just in case
def eval_plots(*datasets, tick_int):
    for i, (data, label, ylabel) in enumerate(datasets):
        for d, l in zip(data, label): 
            ax[i].plot(d, label=l)
        ax[i].set_xlabel('Iterations')
        ax[i].set_ylabel(ylabel)
        ax[i].set_title(f'{ylabel} vs Iterations')
        ax[i].xaxis.set_major_locator(ticker.MultipleLocator(tick_int))
        ax[i].legend()
        
    plt.tight_layout()
    plt.show()




#fig, ax = plt.subplots(1, 3, figsize=(12, 6))

# Plotting with variable names as legend labels
#eval_plots(((nx_lou_m_times, nx_gn_m_times[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Time (s)'),
            #((nx_lou_m_sil, nx_gn_m_sil[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Silhouette Score'),
            #((nx_lou_m_mod, nx_gn_m_mod[:nx_lou_times.shape[1]]), ('NetworkX Louvain', 'NetworkX Girvan Newman'), 'Modularity Score'), 
           #tick_int=1)


# Just Girvan Newman plots
# read nx gn csvs
nx_gn_times = pd.read_csv("data/communities/nx_gn_iteration_times.csv")
nx_gn_m_times = nx_gn_times.mean(axis='rows')

nx_gn_sil = pd.read_csv("data/communities/nx_gn_silhouette_scores_per_iteration.csv")
nx_gn_m_sil = nx_gn_sil.mean(axis='rows')

nx_gn_mod = pd.read_csv("data/communities/nx_gn_modularity_scores_per_iteration.csv")
nx_gn_m_mod = nx_gn_mod.mean(axis='rows')

fig, ax = plt.subplots(3, 1, figsize=(12, 12))

eval_plots(((nx_gn_m_times,), ('NetworkX Girvan Newman',), 'Time (s)'),
            ((nx_gn_m_sil,), ('NetworkX Girvan Newman',), 'Silhouette Score'),
            ((nx_gn_m_mod,), ('NetworkX Girvan Newman',), 'Modularity Score'),
           tick_int=200)
